# 🚀 胸部 X 光分類 - DINOv2 突破 90% 訓練

**目標**: 使用 DINOv2 Vision Transformer 突破 90% Macro-F1

**當前最佳**: 87.574% (Hybrid Adaptive Ensemble)

**預期提升**: +2-4% → **89.5-91.5%**

---

## 📋 訓練計劃

### 選項 A: 快速驗證 (2-3 小時)
- 訓練 Fold 0
- 驗證 DINOv2 效果
- 如果成功 (Val F1 > 88%)，再跑完整 5-Fold

### 選項 B: 完整訓練 (10-15 小時)
- 訓練所有 5 Folds
- 生成集成預測
- 提交到 Kaggle

---

## 🔧 DINOv2 優勢

1. **大規模預訓練**: 142M 圖片自監督學習 (vs ImageNet 1.2M)
2. **Few-shot 學習**: 在小數據集上表現優異
3. **醫學影像適配**: 文獻證實 +2-4% 提升
4. **模型大小**: 86.6M 參數 (ViT-Base)

---

## ⚙️ 環境需求

- **GPU**: T4 (16GB) 或 A100 (40GB)
- **Batch Size**: 8-16 (T4), 24-32 (A100)
- **訓練時間**: 2-3 小時/fold (T4), 1-1.5 小時/fold (A100)
- **總時間**: 10-15 小時 (5-Fold on T4)

---

**開始前請確保**:
1. ✅ 已設置 Kaggle API credentials
2. ✅ 已啟用 GPU Runtime (Runtime → Change runtime type → GPU)
3. ✅ (建議) 掛載 Google Drive 以保存模型


## 1️⃣ 環境檢查與設置

In [ ]:
# 檢查 GPU 可用性
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # 推薦 batch size
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory > 35:  # A100
        recommended_batch = 24
    elif gpu_memory > 14:  # T4/V100
        recommended_batch = 12
    else:
        recommended_batch = 8
    
    print(f"\n✅ 建議 Batch Size: {recommended_batch}")
else:
    print("\n⚠️ WARNING: No GPU found! Please enable GPU in Runtime settings.")
    print("Runtime → Change runtime type → Hardware accelerator → GPU")

## 2️⃣ 安裝依賴

In [ ]:
# 安裝必要的套件
# Colab 已預裝 torch, torchvision, numpy, pandas, Pillow, tqdm, scikit-learn
# 只需安裝 timm (用於 DINOv2) 和 pyyaml

!pip install -q timm pyyaml kaggle

# 驗證安裝
import timm
print(f"✅ timm version: {timm.__version__}")
print(f"✅ DINOv2 models available: {len([m for m in timm.list_models() if 'dinov2' in m])} models")

## 3️⃣ Kaggle API 設置

**兩種方式**:
- **方式 A**: 上傳 `kaggle.json` 文件
- **方式 B**: 手動輸入 username 和 key

In [ ]:
# 方式 A: 上傳 kaggle.json (推薦)
# 1. 從 Kaggle → Account → Create New API Token 下載 kaggle.json
# 2. 運行以下代碼並上傳文件

from google.colab import files
import os

print("請上傳 kaggle.json 文件...")
uploaded = files.upload()

# 設置 Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 驗證
!kaggle competitions list | head -3
print("\n✅ Kaggle API 設置成功！")

In [ ]:
# 方式 B: 手動設置 (如果上傳失敗)
# 取消註釋以下代碼並填入你的 Kaggle credentials

# import json
# os.makedirs('/root/.kaggle', exist_ok=True)
# 
# kaggle_credentials = {
#     "username": "YOUR_KAGGLE_USERNAME",
#     "key": "YOUR_KAGGLE_API_KEY"
# }
# 
# with open('/root/.kaggle/kaggle.json', 'w') as f:
#     json.dump(kaggle_credentials, f)
# 
# !chmod 600 /root/.kaggle/kaggle.json
# print("✅ Kaggle API 設置成功！")

## 4️⃣ Clone GitHub 倉庫

In [ ]:
# Clone 項目倉庫
import os

# 請替換為你的 GitHub 倉庫 URL
GITHUB_REPO = "https://github.com/YOUR_USERNAME/nycu-CSIC30014-LAB3.git"

# 如果是 private repo，需要使用 personal access token
# GITHUB_REPO = "https://YOUR_TOKEN@github.com/YOUR_USERNAME/nycu-CSIC30014-LAB3.git"

if os.path.exists('nycu-CSIC30014-LAB3'):
    print("倉庫已存在，拉取最新代碼...")
    %cd nycu-CSIC30014-LAB3
    !git pull
else:
    print("正在 clone 倉庫...")
    !git clone {GITHUB_REPO}
    %cd nycu-CSIC30014-LAB3

# 驗證
!ls -lh
print("\n✅ 倉庫已準備好！")

## 5️⃣ 下載 Kaggle 競賽數據

In [ ]:
# 下載競賽數據（約 3-4 GB）
!kaggle competitions download -c cxr-multi-label-classification

# 解壓數據
!unzip -q cxr-multi-label-classification.zip -d data/
!rm cxr-multi-label-classification.zip

# 檢查數據結構
print("\n數據集統計:")
!echo "訓練影像: $(ls data/train_images/ 2>/dev/null | wc -l || echo 'N/A')"
!echo "驗證影像: $(ls data/val_images/ 2>/dev/null | wc -l || echo 'N/A')"
!echo "測試影像: $(ls data/test_images/ 2>/dev/null | wc -l || echo 'N/A')"

print("\n✅ 數據下載完成！")

## 6️⃣ 準備 Fold 數據

In [ ]:
# 檢查 Fold CSV 文件是否存在
import pandas as pd

# 檢查是否已有 fold CSV
if os.path.exists('data/fold_0.csv'):
    print("✅ Fold 數據已存在於 GitHub 倉庫中")
    for i in range(5):
        if os.path.exists(f'data/fold_{i}.csv'):
            df = pd.read_csv(f'data/fold_{i}.csv')
            print(f"Fold {i}: {len(df)} 樣本")
else:
    print("⚠️ 警告: Fold 數據不存在")
    print("請確保 GitHub 倉庫中包含 data/fold_0.csv ~ data/fold_4.csv")
    print("或者需要先運行 K-Fold 分割腳本")

# 為每個 fold 創建 train/val CSV (DINOv2 訓練腳本需要)
print("\n創建 train/val CSV 文件...")
for fold in range(5):
    if os.path.exists(f'data/fold_{fold}.csv'):
        # 讀取完整數據
        train_df = pd.read_csv('data/train_data.csv')
        val_df = pd.read_csv('data/val_data.csv')
        all_df = pd.concat([train_df, val_df], ignore_index=True)
        
        # 讀取 fold 分割
        fold_df = pd.read_csv(f'data/fold_{fold}.csv')
        
        # 分割 train/val
        val_indices = fold_df[fold_df['fold'] == fold].index
        train_indices = fold_df[fold_df['fold'] != fold].index
        
        fold_train = all_df.iloc[train_indices].copy()
        fold_val = all_df.iloc[val_indices].copy()
        
        # 添加 source_dir 欄位（指向影像目錄）
        fold_train['source_dir'] = 'data/train_images'
        fold_val['source_dir'] = 'data/val_images'
        
        # 保存
        fold_train.to_csv(f'data/fold{fold}_train.csv', index=False)
        fold_val.to_csv(f'data/fold{fold}_val.csv', index=False)
        
        print(f"Fold {fold}: Train={len(fold_train)}, Val={len(fold_val)}")

print("\n✅ Fold 數據準備完成！")

## 7️⃣ 快速驗證 - 訓練 Fold 0 (選項 A)

**建議**: 先跑 Fold 0 驗證效果，如果 Val F1 > 88%，再跑完整 5-Fold

In [ ]:
# 訓練 Fold 0 (約 2-3 小時 on T4)
# 根據 GPU 調整 batch size

FOLD = 0
EPOCHS = 35
BATCH_SIZE = 12  # T4: 12, A100: 24
IMG_SIZE = 518   # DINOv2 optimal size
LR = 3e-5

!python train_dinov2_breakthrough.py \
    --fold {FOLD} \
    --epochs {EPOCHS} \
    --batch_size {BATCH_SIZE} \
    --img_size {IMG_SIZE} \
    --lr {LR} \
    --output_dir outputs/dinov2_colab

print("\n✅ Fold 0 訓練完成！")
print("請檢查 Val F1 分數:")
!tail -20 outputs/dinov2_colab/fold0/training.log

## 8️⃣ 決策點: 是否繼續完整訓練？

**檢查 Fold 0 結果**:
- ✅ Val F1 ≥ 88% → 繼續跑 Fold 1-4
- ⚠️ Val F1 < 88% → 檢查日誌，調整超參數

In [ ]:
# 查看 Fold 0 最佳結果
import re

log_file = 'outputs/dinov2_colab/fold0/training.log'
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        lines = f.readlines()
    
    # 找到最佳 Val F1
    best_f1 = 0
    for line in lines:
        if 'Best Val F1' in line or 'Val F1' in line:
            match = re.search(r'(\d+\.\d+)%', line)
            if match:
                f1 = float(match.group(1))
                if f1 > best_f1:
                    best_f1 = f1
    
    print(f"\n📊 Fold 0 最佳 Val F1: {best_f1:.2f}%\n")
    
    if best_f1 >= 88:
        print("✅ 結果優秀！建議繼續完整 5-Fold 訓練")
        print("請執行下一個 Cell (選項 B)")
    elif best_f1 >= 86:
        print("⚠️ 結果良好，但未達 88% 目標")
        print("建議: 1) 檢查訓練曲線  2) 調整超參數  3) 或直接跑完整訓練看效果")
    else:
        print("❌ 結果不理想，需要檢查：")
        print("1. 數據是否正確加載")
        print("2. 是否有 GPU 記憶體溢出")
        print("3. 超參數是否需要調整")
else:
    print("⚠️ 找不到訓練日誌，請檢查訓練是否成功完成")

## 9️⃣ 完整訓練 - 所有 5 Folds (選項 B)

**警告**: 需要 10-15 小時 (T4) 或 5-8 小時 (A100)

**建議**: 使用 Colab Pro 避免中斷

In [ ]:
# 訓練所有 5 Folds
# 注意: 這將花費很長時間，建議在 Colab Pro 上運行

import time

EPOCHS = 35
BATCH_SIZE = 12  # 根據 GPU 調整

for fold in range(5):
    print(f"\n{'='*70}")
    print(f"開始訓練 Fold {fold}/4")
    print(f"{'='*70}\n")
    
    start_time = time.time()
    
    !python train_dinov2_breakthrough.py \
        --fold {fold} \
        --epochs {EPOCHS} \
        --batch_size {BATCH_SIZE} \
        --img_size 518 \
        --lr 3e-5 \
        --output_dir outputs/dinov2_colab
    
    elapsed = (time.time() - start_time) / 60
    print(f"\n✅ Fold {fold} 完成！耗時: {elapsed:.1f} 分鐘\n")

print("\n" + "="*70)
print("🎉 所有 5 Folds 訓練完成！")
print("="*70)

## 🔟 生成 5-Fold 集成預測

In [ ]:
# 生成每個 fold 的測試集預測，然後平均
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import timm
from tqdm import tqdm

# 簡化版測試集預測代碼
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, img_dir, img_size=518):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        
        from torchvision import transforms as T
        self.transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        from PIL import Image
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['new_filename'])
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, row['new_filename']

# 載入測試集
test_dataset = TestDataset('data/test_data_sample.csv', 'data/test_images')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 收集所有 fold 的預測
all_preds = []

for fold in range(5):
    model_path = f'outputs/dinov2_colab/fold{fold}/best.pt'
    
    if not os.path.exists(model_path):
        print(f"⚠️ Fold {fold} 模型不存在，跳過")
        continue
    
    # 載入模型
    model = timm.create_model('vit_base_patch14_dinov2.lvd142m', pretrained=False, num_classes=4)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    # 預測
    fold_probs = []
    filenames = []
    
    with torch.no_grad():
        for images, fnames in tqdm(test_loader, desc=f'Fold {fold} 預測'):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            fold_probs.append(probs.cpu().numpy())
            filenames.extend(fnames)
    
    fold_probs = np.concatenate(fold_probs, axis=0)
    all_preds.append(fold_probs)
    print(f"✅ Fold {fold} 預測完成: {fold_probs.shape}")

# 平均所有 fold 的預測
if len(all_preds) > 0:
    avg_probs = np.mean(all_preds, axis=0)
    final_preds = np.argmax(avg_probs, axis=1)
    
    # 創建提交文件
    class_names = ['normal', 'bacteria', 'virus', 'COVID-19']
    submission_df = pd.DataFrame({
        'new_filename': filenames[:len(final_preds)],
        'label': [class_names[p] for p in final_preds]
    })
    
    submission_path = 'submission_dinov2_5fold.csv'
    submission_df.to_csv(submission_path, index=False)
    
    print(f"\n✅ 集成預測已保存: {submission_path}")
    print(f"\n預測分布:")
    print(submission_df['label'].value_counts())
else:
    print("❌ 沒有找到任何訓練好的模型")

## 1️⃣1️⃣ 提交到 Kaggle

In [ ]:
# 提交預測結果到 Kaggle
!kaggle competitions submit \
    -c cxr-multi-label-classification \
    -f submission_dinov2_5fold.csv \
    -m "DINOv2 5-Fold Ensemble - Target 90%+ (Google Colab)"

print("\n✅ 提交完成！")
print("\n查看提交結果:")
!kaggle competitions submissions -c cxr-multi-label-classification | head -5

## 1️⃣2️⃣ 保存結果到 Google Drive (可選)

In [ ]:
# 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 創建保存目錄
save_dir = '/content/drive/MyDrive/CXR_DINOv2_Results'
!mkdir -p {save_dir}

# 複製模型檢查點
!cp -r outputs/dinov2_colab {save_dir}/

# 複製提交文件
!cp submission_dinov2_5fold.csv {save_dir}/

# 複製訓練日誌
!cp -r logs {save_dir}/ 2>/dev/null || echo "No logs directory"

print(f"\n✅ 結果已保存到 Google Drive: {save_dir}")
!ls -lh {save_dir}

## 📊 結果分析與下一步

In [ ]:
# 分析每個 fold 的驗證結果
print("📊 各 Fold 驗證結果摘要:\n")
print(f"{'Fold':<8} {'Val F1':<12} {'狀態':<10}")
print("-" * 40)

fold_results = []
for fold in range(5):
    log_file = f'outputs/dinov2_colab/fold{fold}/training.log'
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            content = f.read()
        
        # 提取最佳 F1
        matches = re.findall(r'Val F1: (\d+\.\d+)%', content)
        if matches:
            best_f1 = max([float(m) for m in matches])
            fold_results.append(best_f1)
            
            status = "✅" if best_f1 >= 88 else "⚠️" if best_f1 >= 86 else "❌"
            print(f"Fold {fold:<4} {best_f1:>6.2f}%     {status}")
        else:
            print(f"Fold {fold:<4} {'N/A':<12} ❌")
    else:
        print(f"Fold {fold:<4} {'未訓練':<12} -")

if fold_results:
    avg_f1 = np.mean(fold_results)
    print("\n" + "=" * 40)
    print(f"平均 Val F1: {avg_f1:.2f}%")
    
    # 預估測試集分數（通常比驗證集低 1-2%）
    estimated_test = avg_f1 - 1.5
    print(f"預估 Test F1: {estimated_test:.2f}%")
    print("=" * 40)
    
    if avg_f1 >= 90:
        print("\n🎉🎉🎉 恭喜！已突破 90% 目標！")
    elif avg_f1 >= 88:
        print("\n✅ 結果優秀！非常接近 90% 目標")
        print("建議: 嘗試更大的 DINOv2 模型 (Large/Giant)")
    else:
        print("\n⚠️ 未達 90% 目標")
        print("建議: 1) 檢查數據增強  2) 調整學習率  3) 嘗試其他方案")

---

## 🎯 完成檢查清單

- [ ] 環境檢查通過（GPU 可用）
- [ ] Kaggle API 設置成功
- [ ] GitHub 倉庫 clone 成功
- [ ] 競賽數據下載完成
- [ ] Fold 數據準備完成
- [ ] Fold 0 快速驗證完成（Val F1 ≥ 88%）
- [ ] 完整 5-Fold 訓練完成
- [ ] 集成預測生成
- [ ] 提交到 Kaggle
- [ ] 結果保存到 Google Drive

---

## 📝 下一步建議

### 如果達到 90%+ ✅
1. 嘗試 DINOv2-Large (更大模型)
2. 與現有最佳模型（87.574%）集成
3. 探索 Test-Time Augmentation (TTA)

### 如果未達標 (< 89%) ⚠️
參考 `BREAKTHROUGH_STRATEGY_ANALYSIS.md` 中的備選方案：
1. **CAPR 偽標籤** (+2-3%) - 類別自適應閾值
2. **ConvNeXt V2** (+0.5-1.5%) - 新一代 CNN
3. **對比學習** (+1.5-2.5%) - 自監督學習

---

**項目文檔**: 詳見 GitHub 倉庫中的 `CLAUDE.md` 和 `README.md`

**當前最佳**: 87.574% (Hybrid Adaptive Ensemble)

**目標**: 突破 90%！
